# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: ** Once the expected output for this supervised learning project is a class label (yes/no) that indicates if the student will pass or not, it's a classification problem. To be a regression problem, the expected output should be a continuous value.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print ("Student data read successfully!")

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [2]:
# TODO: Calculate number of students
n_students = student_data.shape[0]

# TODO: Calculate number of features
n_features = student_data.shape[1] - 1

# TODO: Calculate passing students
n_passed = student_data[student_data['passed'] == 'yes']['passed'].count()

# TODO: Calculate failing students
n_failed = student_data[student_data['passed'] == 'no']['passed'].count()

# TODO: Calculate graduation rate
grad_rate = 100 * n_passed / n_students

# Print the results
print ("Total number of students: {}".format(n_students))
print ("Number of features: {}".format(n_features))
print ("Number of students who passed: {}".format(n_passed))
print ("Number of students who failed: {}".format(n_failed))
print ("Graduation rate of the class: {:.2f}%".format(grad_rate))

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [3]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print ("Feature columns:\n{}".format(feature_cols))
print ("\nTarget column: {}".format(target_col))

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print ("\nFeature values:")
print (X_all.head())

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [4]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print ("Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns)))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [5]:
# TODO: Import any additional functionality you may need here
from sklearn.model_selection import train_test_split

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size = num_test, random_state=55, stratify=y_all)

# Show the results of the split
print ("Training set has {} samples.".format(X_train.shape[0]))
print ("Testing set has {} samples.".format(X_test.shape[0]))

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **

1) Gaussian Naive Bayes:
- Usage example: spam filter;
- Strengths: easy to understand, implement and update; small memory footprint; fast; only requires a small number of training data to estimate the parameters necessary for classification;
- Weaknesses: can't learn interaction between features;
- Performs well: when the features are independent;
- Performs poorly: when the features are dependent;
- characteristics that make the model a good candidate for the problem, based on the given data: more than 50 samples; category prediction; labeled data; less than 100K samples; text data.

    References:
- https://en.wikipedia.org/wiki/Naive_Bayes_classifier
- https://stats.stackexchange.com/questions/23490/why-do-naive-bayesian-classifiers-perform-so-well
- https://www.quora.com/What-are-the-advantages-of-using-a-naive-Bayes-for-classification
- http://scikit-learn.org/stable/tutorial/machine_learning_map/

2) Decision Tree:
- Usage example: disease diagnostic;
- Strengths: simple to understand and interpret; helps determine worst, best and expected values for different scenarios; allows the addition of new possible scenarios; nonlinear relationships between features do not affect tree performance; requires relatively little effort from users for data preparation;
- Weaknesses: a slight change can result in a drastically different tree; without proper pruning or limiting tree growth, they tend to overfit the training data (high accurate output on training data, but low accurate output on test data);
- Performs well: when some measuares are taken to avoid overfit;
- Performs poorly: when it has a lot of uncorrelated features (because it works by finding the interactions between them);
- characteristics that make the model a good candidate for the problem, based on the given data: more than 50 samples; category prediction; labeled data; less than 100K samples; text data.

    References:
- https://en.wikipedia.org/wiki/Decision_tree
- http://www.cbcb.umd.edu/~salzberg/docs/murthy_thesis/survey/node32.html
- http://www.simafore.com/blog/bid/62333/4-key-advantages-of-using-decision-trees-for-predictive-analytics
- https://stats.stackexchange.com/questions/1292/what-is-the-weak-side-of-decision-trees
- https://www.researchgate.net/post/What_is_over_fitting_in_decision_tree
- https://www.quora.com/What-are-the-disadvantages-of-a-decision-tree

3) Support Vector Machines:
- Usage example: face detection;
- Strengths: robust against overfitting; flexibility (many kernels to choose);
- Weaknesses: memory intensive; slow for large data sets; several key parameters that need to be set correctly to achieve the best classification;
- Performs well: on a wide range of classification problems, even problems in high dimensions and that are not linearly separable;
- Performs poorly: when the data sets are very large; there is a lot of noise (can cause some overfitting);
- characteristics that make the model a good candidate for the problem, based on the given data: more than 50 samples; category prediction; labeled data; less than 100K samples; text data.

    References:
- http://www.datasciencecentral.com/profiles/blogs/real-life-applications-of-support-vector-machines
- https://elitedatascience.com/machine-learning-algorithms
- https://br.udacity.com/course/intro-to-machine-learning--ud120
- http://www.nickgillian.com/wiki/pmwiki.php/GRT/SVM


### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [6]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print ("Trained model in {:.4f} seconds".format(end - start))

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print ("Made predictions in {:.4f} seconds.".format(end - start))
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print ("Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train)))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print ("F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train)))
    print ("F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test)))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [7]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A
from sklearn.naive_bayes import GaussianNB
# from sklearn import model_B
from sklearn.tree import DecisionTreeClassifier
# from sklearn import model_C
from sklearn.svm import SVC
# from sklearn import model_D
from sklearn.neighbors import KNeighborsClassifier
# from sklearn import model_E
from sklearn.linear_model import SGDClassifier
# from sklearn import model_F
from sklearn.ensemble import GradientBoostingClassifier
# from sklearn import model_G
from sklearn.linear_model import LogisticRegression
# from sklearn import model_H
from sklearn.neural_network import MLPClassifier

# TODO: Initialize the three models
clf_A = GaussianNB()
clf_B = DecisionTreeClassifier(random_state=86)
clf_C = SVC(random_state=86)
clf_D = KNeighborsClassifier()
clf_E = SGDClassifier(random_state=86)
clf_F = GradientBoostingClassifier(random_state=86)
clf_G = LogisticRegression(random_state=86)
clf_H = MLPClassifier(random_state=86)

# TODO: Set up the training set sizes
# TODO: Execute the 'train_predict' function for each classifier and each training set size
for clf in [clf_A, clf_B, clf_C, clf_D, clf_E, clf_F, clf_G, clf_H]:
    for size in [100, 200, 300]:
        train_predict(clf, X_train[:size], y_train[:size], X_test, y_test)
    print ('='*80)

/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


Training a GaussianNB using a training set size of 100. . .
Trained model in 0.0036 seconds
Made predictions in 0.0011 seconds.
F1 score for training set: 0.8358.
Made predictions in 0.0009 seconds.
F1 score for test set: 0.7556.
Training a GaussianNB using a training set size of 200. . .
Trained model in 0.0029 seconds
Made predictions in 0.0013 seconds.
F1 score for training set: 0.8000.
Made predictions in 0.0011 seconds.
F1 score for test set: 0.7556.
Training a GaussianNB using a training set size of 300. . .
Trained model in 0.0015 seconds
Made predictions in 0.0006 seconds.
F1 score for training set: 0.7990.
Made predictions in 0.0004 seconds.
F1 score for test set: 0.7164.
Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0011 seconds
Made predictions in 0.0003 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0006 seconds.
F1 score for test set: 0.7576.
Training a DecisionTreeClassifier using a training set size of 200. 

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Gaussian Naive Bayes **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0017                  |  0.0005                | 0.8358           | 0.7556          |
| 200               | 0.0012                  |  0.0004                | 0.8000           | 0.7556          |
| 300               | 0.0015                  |  0.0006                | 0.7990           | 0.7164          |

** Classifer 2 - Decision Tree **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0012                  |  0.0003                | 1.0000           | 0.7576          |
| 200               | 0.0019                  |  0.0004                | 1.0000           | 0.6333          |
| 300               | 0.0026                  |  0.0004                | 1.0000           | 0.6500          |

** Classifer 3 - Support Vector Machine **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0020                  | 0.0011                 | 0.8936           | 0.7746          |
| 200               | 0.0048                  | 0.0018                 | 0.8707           | 0.7808          |
| 300               | 0.0095                  | 0.0024                 | 0.8640           | 0.7947          |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **

Classifier 1 - Naive Bayes:
- pros (train/test): good time and F1 Score;
- cons (train/test): should be tuned for a better F1 Score.

Classifier 2 - Decision Tree:
- pros (train/test): good time;
- cons (train/test): bad F1 Score (overfitting); should be tuned to be useful here.

Classifier 3 - Support Vector Machine:
- pros (train/test): good time (for small data set) and F1 Score;
- cons (train/test): should be tuned (with a lot of parameters) for a better F1 Score; tends to a bad time if the data set is increased.

Comparison conclusions:
- time: increasing the training set size, Naive Bayes and Decision Tree tend to be many times faster than Support Vector Machine;
- F1 Score: Support Vector Machine had the best score; Decision Tree, the worst.

So Naive Bayes model seems to be generally the most appropriate based on the available data, limited resources, cost and performance. But Support Vector Machine would be a good or even better choice as well once the data size is small, so the time is not a big deal here.


### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **

Brief explanation:
Naive Bayes is a Machine Learning classifier. Its initial knowledge is based on a set of samples that we need to provide to that. Each sample should have N features and 1 label.
Once the classifier has this knowledge (it was trained), we can provide to that N features and ask for a prediction. So our sample of N features and no label will have a label now. In other words, the sample was classified.

Training Process:
We provide to algorithm a training set with the features and labels. For example:

| Color  | Shape | Size | Fruit |
| :---:  | :---: | :--: | :---: |
| Yellow | Round | 11cm | Orange|
| Yellow | Round | 12cm | Orange|
| Red	 | Round | 10cm | Apple |
| Red    | Round | 11cm | Apple |
| Red    | Round | 12cm | Apple |
| Yellow | Round | 23cm | Melon |
| Yellow | Round | 25cm | Melon |

Prediction:
Given a fruit, it will be classified (label) based on its color, roundness and diameter (features). For example, we know our fruit:
- 1) is round: so it can be classified as an orange, apple or melon;
- 2) has 11cm of diameter: the model don't know a melon of 11cm, so it should be an orange or an apple;
- 3) is yellow: the model don't know a yellow apple, so it should be an orange!

But if the size of our fruit was 30cm?
In this case, the value is not common, it's not in our training set. So it could be an orange or a melon. However, the algorithm has to choose 1 type only and once this size is closer of a melon than an orange, Naive Bayes (and probably any person) would say that our fruit is a melon. So even though our features aren't exactly like a sample of the training set, it still can be classified based on mathematical probabilities.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [8]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

# TODO: Create the parameters list you wish to tune
parameters = {'priors' : [[0.01,0.99],[0.1,0.9],[0.2,0.8],[0.3,0.7],[0.4,0.6],[0.5,0.5],[0.6,0.4],[0.7,0.3],[0.8,0.2],[0.9,0.1],[0.01,0.99]]}

# TODO: Initialize the classifier
clf = GaussianNB()

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(estimator=clf, param_grid=parameters, scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_
print (clf)

# Report the final F1 score for training and testing after parameter tuning
print ("Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train)))
print ("Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test)))

GaussianNB(priors=[0.01, 0.99])
Made predictions in 0.0009 seconds.
Tuned model has a training F1 score of 0.7991.
Made predictions in 0.0005 seconds.
Tuned model has a testing F1 score of 0.7815.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **

In comparison with the untuned model trained with a training set of size 300, we can see a reasonable difference:
- F1 Score for training: 0.7991 x 0.7990
- F1 Score for testing: 0.7815 x 0.7164

So it's possible to conclude that our tuned model is better than the untuned one.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.